In [96]:
import random
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
import numpy
import pandas as pd


In [97]:
stop_words = set(stopwords.words('english')) 

In [149]:
###### Loading Open Job Req file #######
try:    
    Job_details = pd.read_csv ('C:/Users/surendra_dattatrey/Desktop/S&P Data/Gender Neutral JD/Open_req - without_locations.csv', encoding='cp1252')
except Exception as e:
    print(sys.exc_info()[0])
    print(e)
    sys.exit("Please put Job Description file on desired location")

In [150]:
###### Loading Gender score file #######
try:
    gender_score = pd.read_csv(r'C:\Users\surendra_dattatrey\Desktop\S&P Data\Gender Neutral JD\POS_gender_score.csv')
except Exception as e:
    print(sys.exc_info()[0])
    print(e)
    sys.exit("Please put Scoring file on desired location")

In [151]:
df = Job_details 


In [152]:
##### Removing particular string which might occur in JD and not required
for counter in range(len(Job_details)):
    jd_summary = df['Job Description'][counter]
    #def check(string, sub_str): 
    if (jd_summary.find("associated with his or her assessments") != -1): 
        #print("YES") 
        jd_summary = jd_summary.replace('associated with his or her assessments',' ')  
        df['Job Description'][counter] = jd_summary 
        print ("JD found in loop 1")
    #else: 
    #   print("NO")   
    
    if (jd_summary.find("any other characteristic that has no bearing on a person’s ability to perform his or her job") != -1): 
        #print("YES") 
        jd_summary = jd_summary.replace('any other characteristic that has no bearing on a person’s ability to perform his or her job',' ') 
        df['Job Description'][counter] = jd_summary 
        print ("JD found in loop 2")

JD found in loop 2
JD found in loop 2
JD found in loop 2
JD found in loop 2


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [153]:
for i in range(len(Job_details)):
    #print(i)
    data = Job_details['Job Description']
    #data = data.to_frame()
    #data['Job Description']= data['Job Description'].apply(str)
    df['Job_Description'] = pd.DataFrame(data.str.split("\n"))
    df['job_desc_clean'] = df['Job Description'].apply(str).replace('[^a-zA-Z0-9 ]', '', regex=True)
    df['job_desc_clean']= df['job_desc_clean'].str.lower()   
#df

In [154]:
## Word Tokenization process and finding biasness
for x in range(len(Job_details)):
    df['job_description_tokenized'] = df['job_desc_clean'].apply(word_tokenize)
    #df['job_description_tokenized'] = df['job_description_tokenized'].apply(lambda x: [item for item in x if item not in stop_words])  

In [155]:
score_cnt=[]
for a in range(len(Job_details)):
    df['job_description_tokenized_tokens'] = df['job_description_tokenized'].tolist()
    word_lst = df['job_description_tokenized'][a]
    #print(word_lst)
    word_list=[]
    for i in range(len(word_lst)): 
      
        if word_lst[i][0].isalpha():
            word_list.append(word_lst[i])
            
    tagged_tokens=[]
    for token in word_lst:
      
        tagged_tokens.append(nltk.pos_tag([token]))
        #print(token)
    import itertools
    merged = list(itertools.chain(*tagged_tokens))
    
    dfObj = pd.DataFrame(merged) 
    dfObj.columns = ['Token', 'POS']
    
    dfObj_filtered = dfObj[(dfObj.POS == 'NN')| (dfObj.POS == 'NNP')| (dfObj.POS == 'PRP') | (dfObj.POS =='PRP$')]
    df_cd = pd.merge(dfObj_filtered, gender_score, how='inner', on = 'Token')
    df_cd_filter = df_cd[df_cd['Score'] > 0]

    biased_tokens = df_cd_filter['Token']
    len_data = len(df['job_description_tokenized'])
    para_location = []
    for x in range(len(df.index)):
        for a in (df['job_description_tokenized'].iloc[x]):
            #print(x,a)
            for b in biased_tokens: 
                if(a == b):              
                    para_location.append(x) 
    Total = df_cd['Score'].sum()
    score_cnt.append(Total)
    #df['Job_desc_biased_tokens']= list (biased_tokens)
    
    
#score_cnt_df=score_cnt.to_dataFrame()

#df
    

In [156]:
scr_cnt=pd.DataFrame(score_cnt)
df['score_count']=scr_cnt
biased_jd = df[df['score_count']>0]

In [157]:
biased_jd.count()

Job Requisition ID                  0
Job Description                     0
Job_Description                     0
job_desc_clean                      0
job_description_tokenized           0
job_description_tokenized_tokens    0
score_count                         0
dtype: int64

In [158]:
df[df['score_count']>0]

,Job Requisition ID,Job Description,Job_Description,job_desc_clean,job_description_tokenized,job_description_tokenized_tokens,score_count
